In [3]:
import pandas as pd
import numpy as np
import random
import os
import gc

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

train_x = train.drop(['ID', 'Calories_Burned'], axis = 1)
train_y = train['Calories_Burned']
test_x = test.drop('ID', axis = 1)

ordinal_features = ['Weight_Status','Gender']
for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train_x[feature])
    train_x[feature] = le.transform(train_x[feature])
    for label in np.unique(test_x[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_x[feature] = le.transform(test_x[feature])
    
# 히트맵으로 상관관계 표시
plt.figure(figsize = (12,12))
sns.heatmap(train_x.corr(), annot = True, cmap = 'Blues')
plt.show()

rain_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.1, random_state=42, shuffle=True)

train_x # PolynomialFeatures 전처리 이전

poly = PolynomialFeatures()
train_poly = pd.DataFrame(poly.fit_transform(train_x))
val_poly = pd.DataFrame(poly.fit_transform(val_x))
train_poly

model = DecisionTreeRegressor(random_state = 42)

model.fit(train_poly, train_y)

preds = model.predict(val_poly)

mean_squared_error(val_y, preds, squared=False)

KeyError: "['ID', 'Calories_Burned'] not found in axis"